In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
# default_exp traindl

# 用户的点击当做一个序列

# lib导入

In [22]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
# from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import custom_attn
import logging
import matplotlib.pyplot as plt
# import nltk
import numpy as np
import os
import random
%matplotlib inline

In [2]:
import sys
sys.path.append('..')
from mylib.utils.pickle import PickleWrapper

In [3]:
# export
# from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import gc
import tqdm
import os
from tx.config import * 
from tx.eda import * 
from loguru import logger
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)  # 设置显示数据的最大列数，防止出现省略号…，导致数据显示不全
pd.set_option('expand_frame_repr', False)  # 当列太多时不自动换行
# from multiprocessing.dummy import Pool
from sklearn.metrics import classification_report, accuracy_score
import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
trace = logger.add(os.path.join(args.DATA_DIR, 'data_gen/runtime.log'))  

In [5]:
VOCAB_FILE = os.path.join(args.DATA_DIR, 'data_gen/word2vector.model.bin')

In [6]:
# covers about 95% of input data
MAX_SENTS = 91 # maximum number of sentences per document
MAX_WORDS = 26 # maximum number of words per sentence

WORD_EMBED_SIZE = 128
SENT_EMBED_SIZE = 100
DOC_EMBED_SIZE = 50
NUM_CLASSES = 10

BATCH_SIZE = 64
NUM_EPOCHS = 10

In [7]:
wv_from_text = KeyedVectors.load_word2vec_format(VOCAB_FILE, binary=True) 

## Load Vocabulary

In [8]:
words = ['PAD'] + wv_from_text.index2word

word2id = dict(zip(words, range(len(words))))

id2word = {v:k for k, v in word2id.items()}
vocab_size = len(word2id)
print("vocab_size: {:d}".format(vocab_size))

vocab_size: 3412774


In [123]:
# PickleWrapper.dump2File(word2id, os.path.join(args.DATA_DIR, 'data_gen/creativeid2id_dict.pkl'))

## Load w2v Embeddings

In [9]:
E = np.zeros((vocab_size, WORD_EMBED_SIZE))

E[1:] = wv_from_text.vectors

E.shape

(3412774, 128)

In [12]:
# !pip install tensorflow==2.1.0
# -i https://pypi.tuna.tsinghua.edu.cn/simple

In [10]:
def pad_or_truncate(xs, maxlen):
    if len(xs) > maxlen:
        return xs[len(xs) - maxlen:]
#     return ["PAD"] * (maxlen - len(xs)) + xs
    return [0] * (maxlen - len(xs)) + xs

xs = ["The", "cat", "fought", "like", "a", "mouse"]
print(pad_or_truncate(xs, 3))
print(pad_or_truncate(xs, 7))

['like', 'a', 'mouse']
[0, 'The', 'cat', 'fought', 'like', 'a', 'mouse']


In [14]:
# rtest = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/test_creative_id.pkl'))

rtrain = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/train1_creative_id.pkl'))

In [11]:
str_cols = 'user_id age gender'.split()
user_train = pd.read_csv(os.path.join(args.DATA_DIR, 'data_origin/train_preliminary/user.csv'), converters={c: str for c in str_cols}).set_index('user_id')

In [12]:
# gender2id = user_train.gender.map(lambda s: int(s)-1)
age2id = user_train.age.map(lambda s: int(s)-1)

In [65]:
user_train.head()

,age,gender
user_id,,
1,4,1
2,10,1
3,7,2
4,5,1
5,4,1


In [126]:
rtrain[0].head(2)

user_id
1     [[[1], [1], [1], [1], [1], [1], [1], [1], [1],...
10    [[[1], [1], [1], [119124, 1], [1], [1], [18, 1...
dtype: object

# generator

## train_batch_generator

In [17]:
def train_batch_generator(batch_size=1000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    ii = 0
    while True:
        
        # print(f'\ntrain{ii}')
        r = PickleWrapper.loadFromFile(f'/content/train_file/train1_creative_id{ii}.pkl')
        if ii == 43:
            ii = 0
        else:
            ii += 1
        for uu in [r]:
            for i, v in uu.items():
        #             print(v[0])
        #             print('--------')
                # y_gender.append(gender2id[i])
                y_age.append(age2id[i])
                #Shuffle list x in place, and return None.
                
                sents = v[0]
        #             print(sents)

        #             in_queue, out_queue = Manager().Queue(), Manager().Queue()

        #             for sid, words in enumerate(sents):
        #                 in_queue.put((sid, words))
        #             def f(in_queue, out_queue):

        #                 while not in_queue.empty():
        # #                     print(f'in_queue.qsize: {in_queue.qsize()}')
        #                     sid, words = in_queue.get()
        # #                     print(sid)
                for sid, words in enumerate(sents):
                    t_ = words[:-1]
                    if len(t_) > 1:
                      random.shuffle(t_)
                    words = pad_or_truncate(t_+[1], MAX_WORDS)
        #                 print(sid)
        #                 print(words)
        #                     wordsid = pd.Series(words).map(word2id).tolist()
        #                     out_queue.put((sid, words))
        #             pool = Pool(30, f, (in_queue, out_queue))
        #             pool.close()
        #             pool.join()
        #             while not out_queue.empty():
        # #                 print(f'out_queue.qsize: {out_queue.qsize()}')
        #                 sid, wordsid = out_queue.get()
        #                 print(wordsid)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X, to_categorical(y_age, num_classes=NUM_CLASSES)# y_gender, y_age
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_gender = []
                    y_age = []
                    cnt = 0

In [18]:
def val_batch_generator(batch_size=20000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    r = PickleWrapper.loadFromFile(f'/content/train_file/train1_creative_id44.pkl')
    while True:
        for uu in [r]:
            for i, v in uu.items():
                # y_gender.append(gender2id[i])
                y_age.append(age2id[i])
                sents = v[0]
                for sid, words in enumerate(sents):
                    t_ = words[:-1]
                    if len(t_) > 1:
                      random.shuffle(t_)
                    words = pad_or_truncate(t_+[1], MAX_WORDS)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X, to_categorical(y_age, num_classes=NUM_CLASSES)# y_gender, y_age
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_gender = []
                    y_age = []
                    cnt = 0

In [18]:
def val_batch_generator(batch_size=20000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    r = PickleWrapper.loadFromFile(f'/content/train_file/train1_creative_id44.pkl')
    while True:
        for uu in [r]:
            for i, v in uu.items():
                # y_gender.append(gender2id[i])
                y_age.append(age2id[i])
                sents = v[0]
                for sid, words in enumerate(sents):
                    t_ = words[:-1]
                    if len(t_) > 1:
                      random.shuffle(t_)
                    words = pad_or_truncate(t_+[1], MAX_WORDS)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X, to_categorical(y_age, num_classes=NUM_CLASSES)# y_gender, y_age
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_gender = []
                    y_age = []
                    cnt = 0

## test_batch_generator

In [19]:
def test_batch_generator(batch_size=10000):
    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
    y_gender = []
    y_age = []
    cnt = 0
    r = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/test1_creative_id.pkl'))
    cc = 0
    while True:
        for uu in r:
            print(cc)
            cc += 1
            for i, v in uu.items():
#                 y_gender.append(gender2id[i])
#                 y_age.append(age2id[i])
                sents = v[0]
                for sid, words in enumerate(sents):
#                     t_ = words[:-1]
#                     if len(t_) > 1:
#                         random.shuffle(t_)
                    words = pad_or_truncate(t_+[1], MAX_WORDS)
                    X[cnt, sid] = words
                cnt += 1
                if cnt == batch_size:
                    yield X
                    X = np.zeros((batch_size, MAX_SENTS, MAX_WORDS))
                    y_age = []
                    cnt = 0

In [141]:
MAX_SENTS

91

In [142]:
a = batch_generator(rtrain, 10)

In [144]:
for X, y_gender in a:
    print(X.shape)
    print(y_gender)
    break

(10, 91, 20)
[[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


In [109]:
X[1]

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 2.194e+03,
        1.000e+00]])

# build nn 

In [14]:
sent_inputs = Input(shape=(MAX_WORDS,), dtype="int32")
sent_emb = Embedding(input_dim=vocab_size,
                     output_dim=WORD_EMBED_SIZE,
                     weights=[E], 
                     trainable=False,
                     mask_zero=True
                    )(sent_inputs)

sent_enc = Bidirectional(GRU(SENT_EMBED_SIZE,
                            return_sequences=True))(sent_emb)

sent_att = custom_attn.AttentionM()(sent_enc)

sent_model = Model(inputs=sent_inputs, outputs=sent_att)
sent_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 26)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 26, 128)           436835072 
_________________________________________________________________
bidirectional (Bidirectional (None, 26, 200)           138000    
_________________________________________________________________
attention_m (AttentionM)     (None, 200)               226       
Total params: 436,973,298
Trainable params: 138,226
Non-trainable params: 436,835,072
_________________________________________________________________


In [15]:
doc_inputs = Input(shape=(MAX_SENTS, MAX_WORDS), dtype="int32")

doc_emb = TimeDistributed(sent_model)(doc_inputs)

doc_enc = Bidirectional(GRU(DOC_EMBED_SIZE,
                           return_sequences=True))(doc_emb)

doc_att = custom_attn.AttentionM()(doc_enc)
# doc_att = AttentionM()(doc_enc)

fc1_dropout = Dropout(0.2)(doc_att)
fc1 = Dense(50, activation="relu")(fc1_dropout)
fc2_dropout = Dropout(0.2)(fc1)
doc_pred = Dense(NUM_CLASSES, activation="softmax")(fc2_dropout)

model = Model(inputs=doc_inputs, outputs=doc_pred)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 91, 26)]          0         
_________________________________________________________________
time_distributed (TimeDistri (None, 91, 200)           436973298 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 91, 100)           75600     
_________________________________________________________________
attention_m_1 (AttentionM)   (None, 100)               191       
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0   

In [16]:
model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
#                     workers=20,
#                     use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2
880/880 [==============================] - 35240s 40s/step - loss: 1.5755 - accuracy: 0.3581 - val_loss: 1.4462 - val_accuracy: 0.4057
Epoch 2/2
474/880 [===============>..............] - ETA: 4:34:16 - loss: 1.4494 - accuracy: 0.4043

In [22]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
                    workers=20,
#                     use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2
  2/880 [..............................] - ETA: 5:08:24 - loss: 0.6899 - accuracy: 0.5185

UnknownError:  RuntimeError: Your generator is NOT thread-safe. Keras requires a thread-safe generator when `use_multiprocessing=False, workers > 1`. 
Traceback (most recent call last):

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 990, in get
    inputs = self.queue.get(block=True).get()

  File "/root/anaconda3/lib/python3.7/multiprocessing/pool.py", line 657, in get
    raise self._value

  File "/root/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/utils/data_utils.py", line 924, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])

ValueError: generator already executing


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/script_ops.py", 

In [ ]:
history = model.fit(train_batch_generator(rtrain, 1000),
                   steps_per_epoch=880, epochs=2, validation_data=val_batch_generator(rtrain, 2000)
                    , validation_steps=10,
                    workers=20,
                    use_multiprocessing=True
                   )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 880 steps, validate for 10 steps
Epoch 1/2


# predict

In [17]:
model.load_weights(os.path.join(args.DATA_DIR, 'data_gen/model_age_6.weights'))

In [25]:
t = test_batch_generator(1000)

In [26]:
r = model.predict(t, steps=1000)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103


In [56]:
r.shape

(1000000, 10)

In [55]:
np.save(os.path.join(args.DATA_DIR, 'data_gen/rs_age_arr.npy'), r)

In [27]:
r.shape

(1000000, 10)

In [28]:
rs = pd.Series(np.argmax(r, axis=1))

In [29]:
rs.value_counts()

2    357830
4    256373
3    126724
5     99739
1     89325
7     19344
6     15581
0     14944
8     12149
9      7991
dtype: int64

In [30]:
rs.to_pickle(os.path.join(args.DATA_DIR, 'data_gen/rs_age.pkl'))

In [31]:
cc = PickleWrapper.loadFromFile(os.path.join(args.DATA_DIR, 'data_gen/test1_creative_id.pkl'))

In [34]:
cc[0].head()

user_id
3131989    [[[1], [1], [1], [1], [1], [1], [1], [1], [1],...
3131990    [[[1], [1], [1], [1], [973590, 1], [1], [1], [...
3131994    [[[1], [1], [495715, 1], [1], [1], [536353, 12...
3131995    [[[1], [1], [1], [54794, 1159, 1], [1], [1], [...
3132000    [[[1], [1], [1], [6932, 1], [1], [1], [1], [1]...
dtype: object

In [35]:
cs = pd.Series()

In [36]:
for c in cc:
    cs = cs.append(c)

In [37]:
cs.shape

(1000000,)

In [38]:
csf = cs.to_frame()

In [39]:
csf.head()

,0
3131989,"[[[1], [1], [1], [1], [1], [1], [1], [1], [1],..."
3131990,"[[[1], [1], [1], [1], [973590, 1], [1], [1], [..."
3131994,"[[[1], [1], [495715, 1], [1], [1], [536353, 12..."
3131995,"[[[1], [1], [1], [54794, 1159, 1], [1], [1], [..."
3132000,"[[[1], [1], [1], [6932, 1], [1], [1], [1], [1]..."


In [49]:
csf['predicted_age'].value_counts()

3     357830
5     256373
4     126724
6      99739
2      89325
8      19344
7      15581
1      14944
9      12149
10      7991
Name: predicted_age, dtype: int64

In [47]:
csf['predicted_age'] = rs.map(lambda x: str(int(x+1))).tolist()

In [41]:
rs_gender = pd.read_pickle(os.path.join(args.DATA_DIR, 'data_gen/rs_gender.pkl'))

In [50]:
csf['predicted_gender'] = rs_gender.map(lambda x: str(int(x+1))).tolist()

In [51]:
csf['predicted_gender'].value_counts()

1    711621
2    288379
Name: predicted_gender, dtype: int64

In [44]:
csf['predicted_gender']

3131989    NaN
3131990    NaN
3131994    NaN
3131995    NaN
3132000    NaN
          ... 
3131979    NaN
3131983    NaN
3131984    NaN
3131987    NaN
3131988    NaN
Name: predicted_gender, Length: 1000000, dtype: object

In [45]:
rs_gender

0         0
1         1
2         0
3         0
4         1
         ..
999995    0
999996    0
999997    1
999998    0
999999    0
Length: 1000000, dtype: int64

In [52]:
csf['predicted_age predicted_gender'.split()].reset_index().to_csv(os.path.join(args.DATA_DIR, 'data_gen/submission_dl.csv'), index=False)